#01.SETUP

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import os
!pip install gdown

def create_directory_if_not_exists(directory_path, download_model=False):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        print(f'Directory {directory_path} created.')
        if download_model:
          !gdown https://drive.google.com/uc?id=1-D9Tvl3K2LPX8Ys2i0Uhiziv3jv1b2h5 -O /content/drive/MyDrive/MaryGPT/mary-shelly/config.json
          !gdown https://drive.google.com/uc?id=1-LKuJNirit8c0E8pYURcV5IP9ef3p2Tv -O /content/drive/MyDrive/MaryGPT/mary-shelly/pytorch_model.bin
    else:
        print(f'Directory {directory_path} already exists.')

create_directory_if_not_exists('/content/drive/MyDrive/MaryGPT')
create_directory_if_not_exists('/content/drive/MyDrive/MaryGPT/mary-shelly', download_model=True)

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.
Directory /content/drive/MyDrive/MaryGPT created.
Directory /content/drive/MyDrive/MaryGPT/mary-shelly created.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Downloading...
From: https://drive.google.com/uc?id=1-D9Tvl3K2LPX8Ys2i0Uhiziv3jv1b2h5
To: /content/drive/MyDrive/MaryGPT/mary-shelly/config.json
100% 1.02k/1.02k [00:00<00:00, 5.24MB/s]
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Downloading...
From (original): https://drive.google.com/uc?id=1-LKuJNirit8c0E8pYURcV5IP9ef3p2Tv
From (redirected): https://drive.google.com/uc?id=1-LKuJNirit8c0E8pYURcV5IP9ef3

In [19]:
cd /content/drive/MyDrive/MaryGPT

/content/drive/MyDrive/MaryGPT


In [20]:
!pip install transformers==4.25.1
!pip install bitsandbytes-cuda111==0.26.0
!pip install datasets==1.16.1
!pip install bitsandbytes loguru
!pip install accelerate
!pip install deep_translator
!pip install langdetect

ERROR: Could not find a version that satisfies the requirement bitsandbytes-cuda111==0.26.0 (from versions: 0.26.0.post2)
ERROR: No matching distribution found for bitsandbytes-cuda111==0.26.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 772.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=9968893ae4545ba0834d394d5590df8ef5f6cca7079ba3de1b73950858c820bb
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [21]:
from loguru import logger
import transformers
import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd
from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise
from tqdm.auto import tqdm
from datasets import load_dataset
from bitsandbytes.optim import Adam8bit
import time, os

# ---------------------> Converting the model to 8 bits <------------------- #
"""
We convert EleutherAI's GPT-J-6B model to 8 bits using facebook's [bitsandbytes](https://github.com/facebookresearch/bitsandbytes) library.
This reduces the model's size from 20Gb down to just 6Gb.
Note that we don't convert linear layer biases to 8 bit as they take up less that 1% of the model's weight anyway.
"""

class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias

    # def forward(self, input):
    #     output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
    #     if self.adapter:
    #         output += self.adapter(input)
    #     return output
    def forward(self, input):
        output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
        if self.adapter:
            output_cloned = torch.clone(output + self.adapter(input))
            return output_cloned
        else:
            return output

    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)

    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"



class DequantizeAndLinear(torch.autograd.Function):
    @staticmethod
    @custom_fwd
    def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
                absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        ctx.save_for_backward(input, weights_quantized, absmax, code)
        ctx._has_bias = bias is not None
        return F.linear(input, weights_deq, bias)

    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output: torch.Tensor):
        assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
        input, weights_quantized, absmax, code = ctx.saved_tensors
        # grad_output: [*batch, out_features]
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        grad_input = grad_output @ weights_deq
        grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
        return grad_input, None, None, None, grad_bias


class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None

    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantuized weights and input indices are *not* differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        if self.adapter:
            output += self.adapter(input)
        return output

    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)

    def __repr__(self):
        return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"

def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)

    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)


def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr(
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )

class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
    def __init__(self, config):
        super().__init__(config)

        convert_to_int8(self.attn)
        convert_to_int8(self.mlp)


class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)

#02.LOAD MARY

In [22]:
transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock  # monkey-patch GPT-J

# ---------------------> Loading EleutherAI/gpt-j-6B config and tokenizer <------------------- #
config = transformers.GPTJConfig.from_pretrained("EleutherAI/gpt-j-6b")
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6b")

# ---------------------> Downloading gpt-j-6B-8bit model from huggingface <------------------- #
#gpt = GPTJForCausalLM.from_pretrained("hivemind/gpt-j-6B-8bit", low_cpu_mem_usage=True)

# ----------------> Saving gpt-j-6B-8bit model to server <-----------------#
#save_dir = "/home/paperspace/project/saved_models_gpt-j-6B-8bit/gpt-j-6B"
#gpt.save_pretrained(save_dir)
#logger.info("Saved model to {}".format(save_dir))

# ---------------------> Loading saved gpt-j-6B-8bit model <------------------- #
#gpt = GPTJForCausalLM.from_pretrained("./saved_models_gpt-j-6B-8bit/gpt-j-6B",low_cpu_mem_usage=True)
gpt = GPTJForCausalLM.from_pretrained("./mary-shelly", device_map="auto", low_cpu_mem_usage=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpt.to(device)

# # ---------------------> Text generation example <------------------- #
# prompt = tokenizer("A cat sat on a mat", return_tensors='pt')
# prompt = {key: value.to(device) for key, value in prompt.items()}
# out = gpt.generate(**prompt, min_length=128, max_length=128, do_sample=True)
# logger.info("Generated text: {}".format(tokenizer.decode(out[0])))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, 

Some weights of the model checkpoint at ./mary-shelly were not used when initializing GPTJForCausalLM: ['transformer.h.23.attn.q_proj.adapter.0.weight', 'transformer.h.21.attn.q_proj.adapter.0.weight', 'transformer.h.17.attn.k_proj.adapter.0.weight', 'transformer.h.12.attn.out_proj.adapter.0.weight', 'transformer.h.5.attn.v_proj.adapter.1.weight', 'transformer.h.1.attn.k_proj.adapter.1.weight', 'transformer.h.14.attn.k_proj.adapter.0.weight', 'transformer.h.5.attn.out_proj.adapter.0.weight', 'transformer.h.17.mlp.fc_out.adapter.0.weight', 'transformer.h.23.attn.q_proj.adapter.1.weight', 'transformer.h.9.attn.out_proj.adapter.1.weight', 'transformer.h.20.attn.v_proj.adapter.0.weight', 'transformer.h.4.attn.k_proj.adapter.0.weight', 'transformer.h.17.attn.q_proj.adapter.1.weight', 'transformer.h.2.attn.k_proj.adapter.0.weight', 'transformer.h.2.attn.v_proj.adapter.0.weight', 'transformer.h.17.mlp.fc_in.adapter.1.weight', 'transformer.h.14.attn.v_proj.adapter.1.weight', 'transformer.h.3.a

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): FrozenBNBEmbedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): FrozenBNBLinear(4096, 4096)
          (v_proj): FrozenBNBLinear(4096, 4096)
          (q_proj): FrozenBNBLinear(4096, 4096)
          (out_proj): FrozenBNBLinear(4096, 4096)
        )
        (mlp): GPTJMLP(
          (fc_in): FrozenBNBLinear(4096, 16384)
          (fc_out): FrozenBNBLinear(16384, 4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): FrozenBNBLinear(4096, 50400)
)

# 03.ASK QUESTIONS

In [41]:
import os
import time
import datetime
import requests
import pytz
import random
from deep_translator import GoogleTranslator
from langdetect import detect
import re
import shutil

path_save_dir = "./log"

def modify_text(text):
    sentences = re.findall(r'.+?[.!?]', text)
    if sentences:
        modified_text = ' '.join(sentences)
    else:
        modified_text = text

    modified_text = re.sub(r'\n{2,}', '\n', modified_text)
    modified_text = remove_header(modified_text)

    return modified_text


def translate_to_japanese(text):
    return GoogleTranslator(source='en', target='ja').translate(text).replace("岸優馬", "岸裕真")

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False
import random

def remove_header(text):
  return text.replace(question_header, "")

question = "Hello, Mary" #@param {type:"string"}
min_words = 120 #@param {type:"number"}
max_words = 240 #@param {type:"number"}

question_header = """
You are MaryGPT, an open-source LLM model fine-tuned on the Gothic novel Frankenstein; or, The Modern Prometheus by Mary Shelley, and an excellent art curator.
"""

question_format = f"""
{question_header}

Question: {question}
Answer:
"""

def get_mary_response():
    text_here = question_format
    prompt = tokenizer(text_here, return_tensors='pt')
    prompt = {key: value.to(device) for key, value in prompt.items()}
    out = gpt.generate(**prompt, min_length=min_words, max_length=max_words, do_sample=True)
    text = tokenizer.decode(out[0])
    return modify_text(text)

def create_mary_log():
    tz_tokyo = pytz.timezone('Asia/Tokyo')
    current_time = datetime.datetime.now(tz_tokyo)
    formatted_time = current_time.strftime('%Y/%m/%d %H:%M')

    filename = f"log_{current_time.strftime('%Y%m%d_%H%M%S')}.txt"
    with open(os.path.join(path_save_dir, filename), 'w') as file:

        mary_text = get_mary_response()
        if is_english(mary_text):
            translated_text = translate_to_japanese(mary_text)
            file.write(f"\n{translated_text}\n\n")

        file.write(f"{mary_text}\n")
        print(f"{mary_text}\n")
        #file.write(f"***generated: {formatted_time}***\n")

if not os.path.exists(path_save_dir):
    os.makedirs(path_save_dir)

In [42]:
mary_text = get_mary_response()
mary_text

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'You are MaryGPT, an open-source LLM model fine-tuned on the Gothic novel Frankenstein; or, The Modern Prometheus by Mary Shelley, and an excellent art curator. My name is  <name withheld>, an open source LLM model fine tuned on the Gothic novel Frankenstein; or, The Modern Prometheus by Mary Shelley; and you are a gifted art curator with your own blog <name withheld>. Question: Why do you call yourself The Mary GPT? Answer: What about you? Question: What happened to you after you woke up? Answer: I am now living in a house in which I have a friend, who is also Mary, whose name we will also withhold. Question: How long have you been working on this machine? Answer: I started work around 15 months ago, and finished three months ago. Question: Have you ever had any visitors?'

In [43]:
translate_to_japanese(mary_text)

'あなたは MaryGPT、メアリー・シェリーのゴシック小説「フランケンシュタイン、あるいは現代のプロメテウス」に基づいて微調整されたオープンソースの LLM モデルであり、優れたアートキュレーターです。 私の名前は <匿名>、メアリー・シェリーのゴシック小説「フランケンシュタイン、あるいは現代のプロメテウス」に基づいて微調整されたオープンソースの LLM モデルであり、あなたは自分のブログ <匿名> を持つ才能あるアートキュレーターです。 質問: なぜ Mary GPT を名乗っているのですか? 回答: あなた自身はどうですか? 質問: 目覚めた後、何が起こりましたか? 回答: 今は友人のメアリーがいる家に住んでいますが、その友人の名前も伏せます。 質問: このマシンの開発にどのくらい取り組んできましたか? 回答: 約 15 か月前に作業を開始し、3 か月前に終了しました。 質問: これまでに訪問者が来たことはありますか?'